Analysis based on wards

In [1]:
from random import shuffle, randint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, LineString, mapping, shape
from descartes import PolygonPatch
import random
import fiona
import numpy as np
import csv
from fiona import collection

import geopandas as gpd
from geopandas.tools import sjoin # rtree index in-build, used with inner, intersection
import pandas as pd

from collections import defaultdict

In [2]:
def find_intersections(o):
    
    from collections import defaultdict

    paired_ind = [o.index_PP, o.index_WARD]

    d_over_ind = defaultdict(list)

    # creating a dictionary that has prescints as keys and associated small areas as values
    for i in range(len(paired_ind[0].values)):
        if not paired_ind[0].values[i]==paired_ind[1].values[i]: # it shows itself as intersection
            d_over_ind[paired_ind[0].values[i]].append(paired_ind[1].values[i])

    # get rid of the pol precincts with no small areas associated to them- not the most efficient way
    d_temp = {}
    for l in d_over_ind:
        if len(d_over_ind[l]):
            d_temp[l] = d_over_ind[l]

    return d_temp
      
def calculate_join_indices(g1_reind, g2_reind):
       
        # the indexing:
        out = sjoin(g1_reind, g2_reind, how ="inner", op = "intersects")
        
        out.drop('index_right', axis=1, inplace=True) # one index doubles, so we drop one
      
        # output retains only 1 area (left or right join), and gives no intersection area.
        # so we create an array with paired indices: police precincts with a ward
        # we use it in a loop in a function below
        dict_over_ind = find_intersections(out) 
        
        return dict_over_ind
    

In [3]:
def calculate_join(dict_over_ind, g1_reind, g2_reind):
        data_aggreg = []

        for index1, crim in g1_reind.iterrows():
            try:
                index1 = crim.index_PP
                
                wards_found = dict_over_ind[index1]
                for ward in range(len(wards_found)):
                    pom = g2_reind[g2_reind.index_WARD == wards_found[ward]]['geometry']        

                    area_int = pom.intersection(crim['geometry']).area.values[0]
                    area_crim = crim['geometry'].area

                    area_ward = pom.values[0].area

                    popu_ward = g2_reind[g2_reind.index_WARD == wards_found[ward]]['WARD_POP'].values[0]
                    murd_count = crim['murd_cnt']
                    pp_province = crim['province']
                    compnt_nm = crim['compnt_nm']
                    popu_frac = (area_int / area_ward) * popu_ward# fraction of the pop area contained inside the crim
                    
                    extra_info_col_names = ['MUNICNAME', 'PROVINCE', 'WARD_ID']
                        
                    extra_cols = g2_reind[g2_reind.index_WARD == wards_found[ward]][extra_info_col_names]#.filter(regex=("NAME"))

                    data_aggreg.append({'geometry': pom.intersection(crim['geometry']).values[0],\
                                        'index_PP': index1,'index_WARD': wards_found[ward] ,'area_pp': area_crim,\
                                        'area_ward': area_ward,'area_inter': area_int,\
                                        'popu_inter' : popu_frac, 'popu_ward': popu_ward,\
                                        'murd_cnt': murd_count,'province': pp_province,\
                                        'MUNICNAME': extra_cols.MUNICNAME.values[0],\
                                        'PROVINCE': extra_cols.PROVINCE.values[0],\
                                        'WARD_ID': extra_cols.WARD_ID.values[0],\
                                        'PoliceStation': compnt_nm})
            except:
                pass
            
        df_t = gpd.GeoDataFrame(data_aggreg,columns=['geometry', 'index_PP','index_WARD','area_pp',\
                                                     'area_ward','area_inter', 'popu_inter',\
                                                     'popu_ward', 'murd_cnt','province',\
                                                     'MUNICNAME','PROVINCE','WARD_ID', 'PoliceStation'])
        #df_t.to_file(out_name)
        return df_t

In [4]:
# this function adds the remaining columns, calculates fractions etc
def compute_final_col(df_temp):
    # add population data per PP, ratio, etc to the main table
    temp = df_temp.groupby(by=['index_PP'])['popu_inter'].sum().reset_index()

    data_with_population = pd.merge(df_temp, temp, on='index_PP', how='outer')\
            .rename(columns={'popu_inter_y':'popu_frac_per_pp', 'popu_inter_x':'popu_inter'})


    data_with_population['murd_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\
               * data_with_population['murd_cnt']
    data_mur_per_int = data_with_population.groupby(by=['index_WARD'])['murd_per_int'].sum().reset_index()

    data_mur_per_sal = data_mur_per_int.rename(columns={'murd_per_int':'murd_per_ward'})

    data_with_population['scal_fac_per_int'] = data_with_population['popu_inter']/data_with_population['popu_frac_per_pp']\

    data_complete = pd.merge(data_with_population, data_mur_per_sal, on='index_WARD', how='outer')
           
    return data_complete

LOAD the data

In [5]:
ppSHP = 'shapefiles/updated/polPrec_murd2015_prov_aea.shp'
warSHP = '../maps/data/Wards2011_aea.shp'

geo_ward = gpd.GeoDataFrame.from_file(warSHP)
geo_pp = gpd.GeoDataFrame.from_file(ppSHP)

geo_pp_reind = geo_pp.reset_index().rename(columns={'index':'index_PP'})
geo_ward_reind = geo_ward.iloc[:,[2,3,7,8,9]].reset_index().rename(columns={'index':'index_WARD'})
    
dict_int = calculate_join_indices(geo_pp_reind,geo_ward_reind)

In [6]:
# number of wards
len(geo_ward)

4277

In [7]:
geo_pp_reind.head(n=2)

,index_PP,compnt_nm,create_dt,geometry,murd_cnt,province,version
0,0,botshabelo,20140207,POLYGON ((181098.5019976945 -3347212.875774949...,25,Free State,1.1.0
1,1,kubusiedrift,20140207,POLYGON ((278531.9598645054 -3691437.895521421...,0,0,1.1.0


In [8]:
from timeit import default_timer as timer

start = timer() 
df_int = calculate_join(dict_int ,geo_pp_reind, geo_ward_reind)
end = timer()
print("time: ", end - start)  

time:  190.2302980940001


In [9]:
# There are 101,546 intersections 
df_int_aea = compute_final_col(df_int) # add final calculations
df_int_aea.head(n=2)

,geometry,index_PP,index_WARD,area_pp,area_ward,area_inter,popu_inter,popu_ward,murd_cnt,province,MUNICNAME,PROVINCE,WARD_ID,PoliceStation,popu_frac_per_pp,murd_per_int,scal_fac_per_int,murd_per_ward
0,POLYGON ((174410.1256216596 -3347008.790421369...,0,1709,5.539637e+07,17799617.055663,2384089.253057,1127.912782,8421,25,Free State,Mangaung Metropolitan Municipality,Free State,49400028,botshabelo,38323.760000,0.735779,0.029431,5.588438
1,POLYGON ((174407.3500422863 -3347161.563272926...,26,1709,2.809204e+08,17799617.055663,15320373.488126,7248.069705,8421,28,Free State,Mangaung Metropolitan Municipality,Free State,49400028,boithuso,41997.394678,4.832346,0.172584,5.588438


In [10]:
# add column to indicate type of unit
df_int_aea['unit'] = 'ward'

In [11]:
df_int_aea.head(n=2)

,geometry,index_PP,index_WARD,area_pp,area_ward,area_inter,popu_inter,popu_ward,murd_cnt,province,MUNICNAME,PROVINCE,WARD_ID,PoliceStation,popu_frac_per_pp,murd_per_int,scal_fac_per_int,murd_per_ward,unit
0,POLYGON ((174410.1256216596 -3347008.790421369...,0,1709,5.539637e+07,17799617.055663,2384089.253057,1127.912782,8421,25,Free State,Mangaung Metropolitan Municipality,Free State,49400028,botshabelo,38323.760000,0.735779,0.029431,5.588438,ward
1,POLYGON ((174407.3500422863 -3347161.563272926...,26,1709,2.809204e+08,17799617.055663,15320373.488126,7248.069705,8421,28,Free State,Mangaung Metropolitan Municipality,Free State,49400028,boithuso,41997.394678,4.832346,0.172584,5.588438,ward


In [12]:
df_int_aea_waz = df_int_aea.iloc[:,[18,12,11,10,13,6,16,15]].sort('WARD_ID').reset_index().drop('index', axis=1)

In [91]:
#df_int_aea_waz.to_csv('data/wards_pp_intersections_basic.csv')
df_int_aea.to_csv('data/wards_pp_intersections_with_geo.csv')

In [14]:
df_int_aea_waz.head(n=2)

,unit,WARD_ID,PROVINCE,MUNICNAME,PoliceStation,popu_inter,scal_fac_per_int,murd_per_int
0,ward,10101001,Western Cape,Matzikama Local Municipality,vredendal,1128.749984,0.124777,0.998215
1,ward,10101001,Western Cape,Matzikama Local Municipality,lutzville,2068.250016,0.635251,7.623011


Check whether the sums over provinces add up to the official/initial numbers:

In [15]:
df_int_aea.head()
data_prov = df_int_aea[['PROVINCE','province','murd_per_int']]
data_prov.groupby('province')['murd_per_int'].sum()

province
0                   0
Eastern Cape     3051
Free State        943
Gauteng          3671
Kwazulu/Natal    3759
Limpopo           777
Mpumalanga        831
North West        853
Northern Cape     411
Western Cape     3186
Name: murd_per_int, dtype: float64

Adding the remaining crimes based on the PP ID:

In [16]:
def csv_dict_reader(file_obj):

    """
    Read a CSV file using csv.DictReader

    """

    reader = csv.DictReader(file_obj, delimiter=',')
    data_list=[]
    for line in reader:
        data_list.append(line)
        
    return data_list

read in the additional crime data

In [17]:
pp_data=pd.read_csv("data/pp_crime_2015.csv")

In [18]:
pp_data.head(n=2)

,PoliceStation,Crime,Incidents,Province
0,Cape Town Central,Community Reported Crimes,15191,Western Cape
1,Mitchells Plain,Community Reported Crimes,14966,Western Cape


pivot for indexing per police station and crime types as columnd

In [19]:
df_crimes = pp_data.pivot('PoliceStation' ,'Crime','Incidents').reset_index()

replace by lower case to merge with the existing table

In [20]:
df_crimes['PoliceStation'] = df_crimes['PoliceStation'].map(lambda x: x if type(x)!=str else x.lower())

In [21]:
df_crimes.head()

Crime,PoliceStation,Abduction,All theft not mentioned elsewhere,Arson,Assault with the intent to inflict grievous bodily harm,Attempted murder,Attempted sexual offences,Bank robbery,Burglary at non-residential premises,Burglary at residential premises,...,Theft out of or from motor vehicle,Total Contact Crimes,Total Contact Related Crimes,Total Crimes Dependent on Police Action for Detection,Total Other Serious Crimes,Total Property Related Crimes,Total Sexual Offences,Total Subcategories Of Aggravated Robbery,Total TRIO Crimes,Truck hijacking
0,aberdeen,0,38,2,78,2,1,0,18,77,...,10,128,30,41,44,142,11,2,2,0
1,acornhoek,0,117,9,232,12,1,0,115,190,...,43,670,145,202,201,377,138,94,94,0
2,actonville,0,231,7,270,17,2,0,29,162,...,96,688,117,418,285,347,40,84,83,1
3,addo,0,128,10,155,40,1,0,74,162,...,12,399,55,57,133,279,74,30,30,0
4,adelaide,0,60,2,75,1,1,0,33,109,...,11,163,23,43,99,192,17,8,8,0


In [22]:
temp = pd.merge(df_int_aea_waz, df_crimes, on = 'PoliceStation', how ='outer')

In [26]:
temp.columns # 55 column

Index(['unit', 'WARD_ID', 'PROVINCE', 'MUNICNAME', 'PoliceStation',
       'popu_inter', 'scal_fac_per_int', 'murd_per_int', 'Abduction',
       'All theft not mentioned elsewhere', 'Arson',
       'Assault with the intent to inflict grievous bodily harm',
       'Attempted murder', 'Attempted sexual offences', 'Bank robbery',
       'Burglary at non-residential premises',
       'Burglary at residential premises', 'Carjacking', 'Commercial crime',
       'Common assault', 'Common robbery', 'Community Reported Crimes',
       'Contact sexual offences', 'Crimen injuria', 'Culpable homicide',
       'Driving under the influence of alcohol or drugs', 'Drug-related crime',
       'Illegal possession of firearms and ammunition', 'Kidnapping',
       'Malicious injury to property', 'Murder',
       'Neglect and ill-treatment of children', 'Old Indecent assault',
       'Old Rape and attempted rape', 'Public violence', 'Rape',
       'Robbery at non-residential premises',
       'Robbery at r

In [86]:
temp.iloc[:,8:55] = temp.iloc[:,8:55].multiply(temp["scal_fac_per_int"], axis="index")

In [87]:
temp.head()

,unit,WARD_ID,PROVINCE,MUNICNAME,PoliceStation,popu_inter,scal_fac_per_int,murd_per_int,Abduction,All theft not mentioned elsewhere,...,Theft out of or from motor vehicle,Total Contact Crimes,Total Contact Related Crimes,Total Crimes Dependent on Police Action for Detection,Total Other Serious Crimes,Total Property Related Crimes,Total Sexual Offences,Total Subcategories Of Aggravated Robbery,Total TRIO Crimes,Truck hijacking
0,ward,10101001,Western Cape,Matzikama Local Municipality,vredendal,1128.749984,0.124777,0.998215,0,26.327933,...,5.739739,74.367052,12.103362,29.322579,40.178172,42.798488,7.736170,0.748662,0.748662,0
1,ward,10101002,Western Cape,Matzikama Local Municipality,vredendal,725.543920,0.080205,0.641638,0,16.923209,...,3.689420,47.802049,7.779864,18.848123,25.825939,27.510240,4.972696,0.481229,0.481229,0
2,ward,10101003,Western Cape,Matzikama Local Municipality,vredendal,3266.950650,0.361143,2.889143,0,76.201159,...,16.612575,215.241189,35.030865,84.868590,116.288025,123.872026,22.390862,2.166858,2.166858,0
3,ward,10101004,Western Cape,Matzikama Local Municipality,vredendal,3109.672670,0.343757,2.750054,0,72.532673,...,15.812810,204.879018,33.344404,80.782834,110.689671,117.908563,21.312918,2.062540,2.062540,0
4,ward,10101005,Western Cape,Matzikama Local Municipality,vredendal,797.848971,0.088198,0.705582,0,18.609714,...,4.057094,52.565826,8.555176,20.726458,28.399658,30.251810,5.468257,0.529186,0.529186,0


In [83]:
len(df_int_aea_waz_full)

11779

In [88]:
df_int_aea_waz_full = temp

In [89]:
df_int_aea_waz_full.to_csv('data/wards2011_pp_intersections_full.csv')

In [90]:
df_int_aea_waz_full.head()

,unit,WARD_ID,PROVINCE,MUNICNAME,PoliceStation,popu_inter,scal_fac_per_int,murd_per_int,Abduction,All theft not mentioned elsewhere,...,Theft out of or from motor vehicle,Total Contact Crimes,Total Contact Related Crimes,Total Crimes Dependent on Police Action for Detection,Total Other Serious Crimes,Total Property Related Crimes,Total Sexual Offences,Total Subcategories Of Aggravated Robbery,Total TRIO Crimes,Truck hijacking
0,ward,10101001,Western Cape,Matzikama Local Municipality,vredendal,1128.749984,0.124777,0.998215,0,26.327933,...,5.739739,74.367052,12.103362,29.322579,40.178172,42.798488,7.736170,0.748662,0.748662,0
1,ward,10101002,Western Cape,Matzikama Local Municipality,vredendal,725.543920,0.080205,0.641638,0,16.923209,...,3.689420,47.802049,7.779864,18.848123,25.825939,27.510240,4.972696,0.481229,0.481229,0
2,ward,10101003,Western Cape,Matzikama Local Municipality,vredendal,3266.950650,0.361143,2.889143,0,76.201159,...,16.612575,215.241189,35.030865,84.868590,116.288025,123.872026,22.390862,2.166858,2.166858,0
3,ward,10101004,Western Cape,Matzikama Local Municipality,vredendal,3109.672670,0.343757,2.750054,0,72.532673,...,15.812810,204.879018,33.344404,80.782834,110.689671,117.908563,21.312918,2.062540,2.062540,0
4,ward,10101005,Western Cape,Matzikama Local Municipality,vredendal,797.848971,0.088198,0.705582,0,18.609714,...,4.057094,52.565826,8.555176,20.726458,28.399658,30.251810,5.468257,0.529186,0.529186,0
